In [6]:
import spacy_annotator as spa
import spacy
import pandas as pd
from faker import Faker
import random
import json

In [22]:
fake = Faker()

data = []
for _ in range(10):
    record = {
        "text": f"""Medical Report 
Patient Information: 
• Name: {fake.name()} 
• Address: {fake.address()}, {fake.country()}
• Date of Birth: {fake.date_of_birth(minimum_age=18, maximum_age=100).strftime('%B %d, %Y')} 
• ID Number: {''.join(random.choices('0123456789', k=9))} 
• Insurance Number: {''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890', k=12))} 
• Email: {fake.email()} 
• Credit Card Number: {''.join(random.choices('0123456789', k=4))}-{''.join(random.choices('0123456789', k=4))}-{''.join(random.choices('0123456789', k=4))}-{''.join(random.choices('0123456789', k=4))} 
Date of Consultation: {fake.date_time_between(start_date="-5y", end_date="-1d").strftime('%B %d, %Y')}"""
    }
    data.append(record)

In [24]:
for record in data:
    # Print the text with the desired format
    print(record['text'])
    print()

Medical Report 
Patient Information: 
• Name: Ashley Dennis 
• Address: 6438 Kristopher Mill Apt. 400
Grosschester, GA 68049, Tokelau
• Date of Birth: July 18, 1990 
• ID Number: 216200768 
• Insurance Number: FMFEXS1W0942 
• Email: rrhodes@example.com 
• Credit Card Number: 0966-2129-3896-2298 
Date of Consultation: October 29, 2022

Medical Report 
Patient Information: 
• Name: Daniel Carpenter 
• Address: 38393 Vanessa Road Apt. 246
Johnstonhaven, NY 42789, Ghana
• Date of Birth: August 13, 1946 
• ID Number: 004301408 
• Insurance Number: C1BIOK5EWUPF 
• Email: katelyncase@example.org 
• Credit Card Number: 9717-8347-0549-5733 
Date of Consultation: December 04, 2021

Medical Report 
Patient Information: 
• Name: Tara Davis 
• Address: USNV Moore
FPO AE 14864, Palestinian Territory
• Date of Birth: October 07, 1986 
• ID Number: 125038689 
• Insurance Number: 64VN1Q9A0UX2 
• Email: bellangel@example.net 
• Credit Card Number: 7945-9423-6480-7753 
Date of Consultation: October 08, 2

In [ ]:
### Specific format is required for fine trainning

In [7]:
with open("Annotations_Full.json", "r") as f:
    data = json.load(f)

trainning = data["annotations"]
trainning = [tuple(i) for i in trainning]

for entry in trainning:
        entry[1]["entities"][0] = tuple(entry[1]["entities"][0])

trainning[0][1]

{'entities': [(46, 59, 'PERSON'),
  [72, 133, 'GPE'],
  [151, 164, 'DATE'],
  [179, 188, 'ID'],
  [210, 222, 'INSURANCE'],
  [233, 252, 'EMAIL'],
  [276, 295, 'CREDIT_CARD'],
  [319, 335, 'DATE'],
  [504, 510, 'PERSON'],
  [1111, 1117, 'PERSON'],
  [2272, 2278, 'PERSON'],
  [2810, 2816, 'PERSON'],
  [3050, 3070, 'PERSON'],
  [3103, 3122, 'GPE']]}

In [2]:
#Names
#Address
#Credit card Number
#Email
#date of birth (only year perhaps?)
#ID number
#Insurance Number

In [9]:
# import argilla as rg
# from argilla._constants import DEFAULT_API_KEY

# # Replace api_url with the url to your HF Spaces URL if using Spaces
# # Replace api_key if you configured a custom API key
# rg.init(
#     api_url="http://localhost:6900",
#     api_key = DEFAULT_API_KEY 
# )


In [19]:
import fitz
pdfIn = fitz.open("PDF/2.pdf")
for page in pdfIn:
    text = page.get_text() 
    print(text)

Father deceased due to myocardial infarction age 60.
Mother alive, with a history of hypertension and diabetes.
No siblings.
Social History: Mr. Smith is employed as a software engineer and reports a sedentary
lifestyle. He denies tobacco or recreational drug use. He consumes alcohol occasionally,
socially.
PAST MEDICAL HISTORY
· Hypertension, diagnosed 5 years ago.
· Allergic rhinitis, managed with over the counter antihistamines.
· No history of significant surgical procedures.
DATE OF CONSULTATION: FEBRUARY 18, 2024
FAMILY  HISTORY
CHIEF COMPLAINT: THE PATIENT PRESENTS WITH COMPLAINTS OF PERSISTENT
HEADACHES AND FATIGUE OVER THE PAST MONTH.
HISTORY OF PRESENT ILLNESS: MR. SMITH REPORTS EXPERIENCING FREQUENT
HEADACHES, PARTICULARLY IN THE FRONTAL REGION, ACCOMPANIED BY FEELINGS OF
FATIGUE AND OCCASIONAL DIZZINESS. HE NOTES THAT THE SYMPTOMS HAVE BEEN
PROGRESSIVELY WORSENING OVER THE PAST FEW WEEKS AND ARE NOW INTERFERING
WITH HIS DAILY ACTIVITIES.
DR. EMILY JOHNSON, M.D.
BOARD CERTIF

In [2]:
import fitz
import spacy


def extract_names_and_addresses(doc):
    Names = list()
    location = list()
    date = list()
    ID = list()
    for ent in doc.ents:
        # if ent.label_ == "GPE": #or ent.label_ == "GPE" or ent.label_ == "DATE":
        # 
        if ent.label_ == "PERSON" and ent.text not in Names: # Filter for names and geopolitical entities (which often contain addresses)
            Names.append(ent.text)

        if ent.label_ == "GPE":
            location.append(ent.text)

        if ent.label == "DATE":
            date.append(ent.text)

        if ent.label == "ID":
            ID.append(ent.text)

    Names_cleared = set([word for phrase in Names for word in phrase.split()])
    #location
    return Names + location + date + ID

# nlp = spacy.load("en_core_web_trf")
# nlp = spacy.load("en_core_web_lg")
# spacy.load("en_core_web_lg")
nlp =  spacy.load("AI_fine_tuning/model-best")
pdfIn = fitz.open("PDF/draft22.pdf")
mask = False




for page in pdfIn:
    text = page.get_text() 
    doc = nlp(text)
    spacy.displacy.render(doc, style="ent")
    entities = [(ent.label_, ent.start_char, ent.end_char) for ent in doc.ents]
    tokens = [token.text for token in doc]
    print(tokens)
    # continue
    # print(text)
    # target_text = extract_names_and_addresses(doc)
    # print(target_text)
    # text_instances = [page.search_for(text) for text in target_text] 
    

    # # iterate through each instance for highlighting
    # for inst in text_instances:
    #     annot = page.add_highlight_annot(inst)

    #     if mask == True:
    #         annot.set_colors({"stroke":(0, 0, 0)})


    #     annot.update()


# Saving the PDF Output
pdfIn.save("Medical_Report_masked.pdf")



['Medical', 'Report', '\n', 'Patient', 'Information', ':', '\n ', 'Name', ':', 'John', 'Smith', '\n             ', 'Address', ':', '725', '5th', 'Ave', ',', 'New', 'York', ',', 'NY', '10022', ',', 'USA', '.', '\n             ', 'Date', 'of', 'Birth', ':', 'January', '1', ',', '1980', '\n             ', 'ID', 'Number', ':', '987654321', '\n ', 'Insurance', 'Number', ':', 'INS123456789', '\n             ', 'Email', ':', 'john.smith@example.com', '\n ', 'Credit', 'Card', 'Number', ':', '6592', '-', '1223', '-', '4857', '-', '1234', '\n', 'Date', 'of', 'Consultation', ':', 'February', '18', ',', '2024', '\n', 'Chief', 'Complaint', ':', 'The', 'patient', 'presents', 'with', 'complaints', 'of', 'persistent', 'headaches', 'and', 'fatigue', 'over', '\n', 'the', 'past', 'month', '.', '\n', 'History', 'of', 'Present', 'Illness', ':', 'Mr.', 'Smith', 'reports', 'experiencing', 'frequent', 'headaches', ',', 'particularly', 'in', 'the', '\n', 'frontal', 'region', ',', 'accompanied', 'by', 'feelings

['Social', 'History', ':', 'Mr.', 'Smith', 'is', 'employed', 'as', 'a', 'software', 'engineer', 'and', 'reports', 'a', 'sedentary', 'lifestyle', '.', 'He', '\n', 'denies', 'tobacco', 'or', 'recreational', 'drug', 'use', '.', 'He', 'consumes', 'alcohol', 'occasionally', ',', 'socially', '.', '\n', 'Review', 'of', 'Systems', ':', '\n', '\uf0b7', 'Cardiovascular', ':', 'No', 'chest', 'pain', 'or', 'palpitations', '.', '\n', '\uf0b7', 'Respiratory', ':', 'No', 'cough', 'or', 'shortness', 'of', 'breath', '.', '\n', '\uf0b7', 'Gastrointestinal', ':', 'Occasional', 'heartburn', ',', 'otherwise', 'unremarkable', '.', '\n', '\uf0b7', 'Neurological', ':', 'Headaches', 'as', 'described', 'above', '.', '\n', '\uf0b7', 'Musculoskeletal', ':', 'No', 'joint', 'pain', 'or', 'swelling', '.', '\n', '\uf0b7', 'Dermatological', ':', 'No', 'rashes', 'or', 'lesions', '.', '\n', 'Physical', 'Examination', ':', '\n', '\uf0b7', 'Vital', 'Signs', ':', 'Blood', 'pressure', '130/80', 'mmHg', ',', 'pulse', '72', '

['\uf0b7', 'Lifestyle', 'modifications', 'including', 'stress', 'reduction', 'techniques', 'and', 'regular', 'physical', 'activity', '.', '\n', '\uf0b7', 'Follow', '-', 'up', 'appointment', 'scheduled', 'in', 'two', 'weeks', 'to', 'assess', 'response', 'to', 'treatment', 'and', '\n', 'consider', 'additional', 'interventions', 'as', 'necessary', '.', '\n', 'Follow', '-', 'up', 'Instructions', ':', '\n', '\uf0b7', 'Mr.', 'Smith', 'is', 'advised', 'to', 'monitor', 'his', 'blood', 'pressure', 'at', 'home', 'regularly', 'and', 'to', 'report', 'any', '\n', 'significant', 'changes', 'or', 'new', 'symptoms', 'promptly', '.', ' \n', '\uf0b7', 'He', 'is', 'counseled', 'on', 'lifestyle', 'modifications', 'including', 'a', 'heart', '-', 'healthy', 'diet', ',', 'regular', 'exercise', ',', '\n', 'and', 'smoking', 'cessation', 'if', 'applicable', '.', '\n \n \n', 'Dr.', 'Emily', 'Johnson', ',', 'M.D.', '\n', 'Board', 'Certified', 'Physician', '\n', 'Mayo', 'Clinic', '\n \n \n \n \n']


In [ ]:
#Input data assessement
#output data asseement

#data gov part. (not AI)
#where data was obtained : Trainning/testing.
#model data assessement